# zib gereed maken voor mapping

* inlezen en layout consistent maken met XtEHR
* opslaan voor mapping - zie mapping Notebook

## imports

In [21]:
import pandas as pd
import os

## toon de zibs in folder (hier: zib2024)

In [26]:
base = 'zibs2024'
folder = os.path.join(base, 'downloads')
files = os.listdir(folder)
files

['Nl.zorg.MedicalDevice-v5.0(2024EN).xlsx',
 'Nl.zorg.part.AddressInformation-v1.2(2024EN).xlsx',
 'Nl.zorg.part.ContactInformation-v1.3.1(2024EN).xlsx',
 'Nl.zorg.part.NameInformation-v1.2(2024EN).xlsx',
 'Nl.zorg.Patient-v4.3(2024EN).xlsx',
 'Nl.zorg.Procedure-v6.0(2024EN).xlsx']

## kies een zib

In [39]:
file = files[5]
print(file)
url = os.path.join(folder, file)
zibdf = pd.read_excel(url, sheet_name='Data', header=1)
zibdf

Nl.zorg.Procedure-v6.0(2024EN).xlsx


C:\Users\edelman_nict1\AppData\Roaming\Python\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Concept,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Alias,Type,Card.,Stereotype,Id,Definition,DefinitionCode,Reference,Constraints
0,NaN,Procedure,NaN,NaN,NaN,NaN,NaN,NL: Verrichting,NaN,NaN,rootconcept,NL-CM:14.1.1,Root concept of the Procedure information mode...,SNOMED CT: 71388002 Procedure,NaN,NaN
1,NaN,NaN,ProcedureStartDate,NaN,NaN,NaN,NaN,NL: VerrichtingStartDatum,TS,0..1,data,NL-CM:14.1.2,The (desired) start date (and if possible star...,NaN,NaN,NaN
2,NaN,NaN,ProcedureEndDate,NaN,NaN,NaN,NaN,NL: VerrichtingEindDatum,TS,0..1,data,NL-CM:14.1.3,The end date (and if possible end time) of the...,NaN,NaN,NaN
3,NaN,NaN,Performer::HealthProfessional,NaN,NaN,NaN,NaN,NL: Uitvoerder::Zorgverlener,NaN,0..*,"context,reference",NL-CM:14.1.6,The healthcare professional who carried out or...,NaN,This is a reference to the rootconcept of inf...,NaN
4,NaN,NaN,Location::HealthcareProvider,NaN,NaN,NaN,NaN,NL: Locatie::Zorgaanbieder,NaN,0..1,"context,reference",NL-CM:14.1.5,The healthcare provider where the procedure wa...,NaN,This is a reference to the rootconcept of info...,NaN
5,NaN,NaN,ProcedureAnatomicalLocation::AnatomicalLocation,NaN,NaN,NaN,NaN,NL: VerrichtingAnatomischeLocatie::Anatomische...,NaN,0..1,"data,reference",NL-CM:14.1.13,Anatomical location that is the focus of the p...,SNOMED CT: 405813007 Procedure site - Direct,This is a reference to the rootconcept of info...,NaN
6,NaN,NaN,Indication,NaN,NaN,NaN,NaN,NL: Indicatie,NaN,0..*,container,NL-CM:14.1.14,Container of the Indication concept.This conta...,NaN,NaN,NaN
7,NaN,NaN,NaN,Diagnosis,NaN,NaN,NaN,NL: Diagnose,NaN,(0..1),"data,reference",NL-CM:14.1.15,A diagnosis that serves as an indication for t...,NaN,This is a reference to the rootconcept of info...,NaN
8,NaN,NaN,NaN,Reaction,NaN,NaN,NaN,NL: Reactie,NaN,(0..1),"data,reference",NL-CM:14.1.16,An undesired reaction to a substance or radiat...,NaN,This is a reference to the rootconcept of info...,NaN
9,NaN,NaN,NaN,Symptom,NaN,NaN,NaN,NL: Symptoom,NaN,(0..1),"data,reference",NL-CM:14.1.17,A symptom that serves as an indication for the...,NaN,This is a reference to the rootconcept of info...,NaN


## opschonen zib

* lowercase kolomnamen
* paar labels laten vervallen
* name en path kolom toevoegen
* name vullen (naam attribuut)
* kolommen opschonen tot benodigde

Handmatig:
* path vullen (als in logical model, dus pad.pad2.pad3.naam etc.)

In [40]:
zibdf.columns = [col.lower() for col in zibdf.columns]
zibdf = zibdf.drop(labels=['unnamed: 0', 'reference'], axis=1).dropna(how='all')
zibdf.insert(0, 'name', None)
zibdf.insert(0, 'path', None)
zibdf.fillna('')
concepts = []
for i, row in zibdf.iterrows():
    # Loop thru columns 2 tot 8 (Concept and 'Unnamed' ones)
    for j in range(2, 6):
        # print(j, row.iloc[j])
        if not pd.isna(row.iloc[j]):
            # Remove last element if next column is empty
            if len(concepts) > j -2:
                concepts.pop()
            try:
                concepts[j - 2] = row.iloc[j]
            except IndexError:
                concepts.append(row.iloc[j])
    # print(i, concepts)
    zibdf.at[i, 'path'] = '.'.join(concepts)
    zibdf.at[i, 'name'] = concepts[-1]
zibdf = zibdf[['name', 'path', 'alias', 'type', 'card.', 'stereotype', 'id', 'definition', 'definitioncode']].fillna('')
zibdf

,name,path,alias,type,card.,stereotype,id,definition,definitioncode
0,Procedure,Procedure,NL: Verrichting,,,rootconcept,NL-CM:14.1.1,Root concept of the Procedure information mode...,SNOMED CT: 71388002 Procedure
1,ProcedureStartDate,Procedure.ProcedureStartDate,NL: VerrichtingStartDatum,TS,0..1,data,NL-CM:14.1.2,The (desired) start date (and if possible star...,
2,ProcedureEndDate,Procedure.ProcedureEndDate,NL: VerrichtingEindDatum,TS,0..1,data,NL-CM:14.1.3,The end date (and if possible end time) of the...,
3,Performer::HealthProfessional,Procedure.Performer::HealthProfessional,NL: Uitvoerder::Zorgverlener,,0..*,"context,reference",NL-CM:14.1.6,The healthcare professional who carried out or...,
4,Location::HealthcareProvider,Procedure.Location::HealthcareProvider,NL: Locatie::Zorgaanbieder,,0..1,"context,reference",NL-CM:14.1.5,The healthcare provider where the procedure wa...,
5,ProcedureAnatomicalLocation::AnatomicalLocation,Procedure.ProcedureAnatomicalLocation::Anatomi...,NL: VerrichtingAnatomischeLocatie::Anatomische...,,0..1,"data,reference",NL-CM:14.1.13,Anatomical location that is the focus of the p...,SNOMED CT: 405813007 Procedure site - Direct
6,Indication,Procedure.Indication,NL: Indicatie,,0..*,container,NL-CM:14.1.14,Container of the Indication concept.This conta...,
7,Diagnosis,Procedure.Indication.Diagnosis,NL: Diagnose,,(0..1),"data,reference",NL-CM:14.1.15,A diagnosis that serves as an indication for t...,
8,Reaction,Procedure.Indication.Reaction,NL: Reactie,,(0..1),"data,reference",NL-CM:14.1.16,An undesired reaction to a substance or radiat...,
9,Symptom,Procedure.Indication.Symptom,NL: Symptoom,,(0..1),"data,reference",NL-CM:14.1.17,A symptom that serves as an indication for the...,


## Excel opslaan

In [41]:
url = os.path.join(base, 'mappable', file)
zibdf.to_excel(url, index=False)